# Step 2: Private Prediction using Syft Keras (without queuing)

In [1]:
import tensorflow as tf
import numpy as np
import syft
import tf_encrypted as tfe

In [2]:
#import tf_encrypted as tfe
from syft.keras.model import Sequential
from syft.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

In [3]:
task_classes = 10
task_shape = [1, 28, 28, 1]
pre_trained_weights = 'short-conv-mnist.h5'

In [4]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_size=1, input_shape=task_shape[1:]))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(task_classes, name="logit"))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# Load pre-trained weights
model.load_weights(pre_trained_weights)

In [6]:
data = np.ones(task_shape)

In [7]:
public_prediction = model.predict(data)
print("Public Prediction:", public_prediction)

Public Prediction: [[  6.3294497  -9.31974     6.433522   -0.1688543  -4.8526316  -2.10865
   10.5845375 -10.482777    3.7927673  -4.036223 ]]


### Secure the model by sharing the weights

In [8]:
prot = tfe.protocol.SecureNN()

tfe_graph, tfe_model = model.share(prot=prot)

INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


conv2d_input InputLayer
conv2d Conv2D
average_pooling2d AveragePooling2D
activation Activation


INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


conv2d_1 Conv2D
average_pooling2d_1 AveragePooling2D
activation_1 Activation


INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


conv2d_2 Conv2D
average_pooling2d_2 AveragePooling2D
activation_2 Activation
flatten Flatten
logit Dense


In [9]:
with tfe_graph.as_default():

    with prot:
        x = tfe.define_private_placeholder(task_shape)
        y = tfe_model(x)

    initializer = tf.global_variables_initializer()

In [10]:
with tfe.Session(graph=tfe_graph) as sess:
    sess.run(initializer)
    actual = sess.run(y.reveal(), feed_dict=x.feed(data))

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


In [11]:
print("Private Prediction: ", actual)

Private Prediction:  [[ 6.35604329e+00 -9.28989483e+00  6.39308032e+00  3.35314739e-03
  -4.85611949e+00 -2.10912971e+00  1.01771071e+01 -1.02007316e+01
   3.81694864e+00 -4.08931565e+00]]
